Pytorch is a popular deep learning framework and it's easy to get started.

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

BATCH_SIZE = 128
NUM_EPOCHS = 10
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
print(torch.cuda.get_device_name(0))
print(torch.rand(3,3).cuda()) 

cuda:0
GeForce GTX 1050 Ti
tensor([[0.4648, 0.9907, 0.0988],
        [0.9436, 0.1117, 0.2777],
        [0.8218, 0.1152, 0.7450]], device='cuda:0')


First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [4]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = nn.functional.relu(out)
        out = nn.functional.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = nn.functional.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = nn.functional.relu(out)
        out = self.fc2(out)
        out = nn.functional.log_softmax(out,dim=1)
        return out

    
model = SimpleNet().to(DEVICE)
# TODO:define loss function and optimiter
#criterion = 
optimizer = torch.optim.Adam(model.parameters());

Next, we can start to train and evaluate!

Q5:

Please print the training and testing accuracy.

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%150 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    # evaluate
    # TODO:calculate the accuracy using training and testing dataset
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()       
    print('\nTrain set:  Accuracy: {}/{} ({:.2f}%)\n'.format(
             correct, len(train_loader.dataset),
            100. * correct / len(train_loader.dataset)))
    
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()       
    print('\nTest set:  Accuracy: {}/{} ({:.2f}%)\n'.format(
             correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))        


Train Epoch: 0 [19072/60000 (32%)]	Loss: 0.121886
Train Epoch: 0 [38272/60000 (64%)]	Loss: 0.051665
Train Epoch: 0 [57472/60000 (96%)]	Loss: 0.124753

Train set:  Accuracy: 58862/60000 (98.10%)


Test set:  Accuracy: 9803/10000 (98.03%)

Train Epoch: 1 [19072/60000 (32%)]	Loss: 0.047954
Train Epoch: 1 [38272/60000 (64%)]	Loss: 0.122968
Train Epoch: 1 [57472/60000 (96%)]	Loss: 0.024701

Train set:  Accuracy: 59273/60000 (98.79%)


Test set:  Accuracy: 9868/10000 (98.68%)

Train Epoch: 2 [19072/60000 (32%)]	Loss: 0.078689
Train Epoch: 2 [38272/60000 (64%)]	Loss: 0.038244
Train Epoch: 2 [57472/60000 (96%)]	Loss: 0.113636

Train set:  Accuracy: 59238/60000 (98.73%)


Test set:  Accuracy: 9842/10000 (98.42%)

Train Epoch: 3 [19072/60000 (32%)]	Loss: 0.013801
Train Epoch: 3 [38272/60000 (64%)]	Loss: 0.005844
Train Epoch: 3 [57472/60000 (96%)]	Loss: 0.003286

Train set:  Accuracy: 59493/60000 (99.16%)


Test set:  Accuracy: 9863/10000 (98.63%)

Train Epoch: 4 [19072/60000 (32%)]	Loss: 0.03368